## Example on how to use the GMS compilation algorithm

Here we show how to use the compilation algorithm(s) developed in https://arxiv.org/abs/2507.20694.
The example circuit we are going to use is the `adder_n28` circuit from [QASMBench](https://github.com/pnnl/QASMBench).

We begin by cleaning up the circuit, possibly removing the assembly instructions for barriers and measurements.

In [1]:
# All necessary imports
import sys  
sys.path.append('..')
from benchmark_utils import remove_meas_and_barriers, calculate_circuit_runtime, load_circuit, save_recompiled
from synthesis_algorithms import extract_peephole_ILP, extract_peephole_only
import qiskit.qasm2
from qiskit import transpile

In [2]:
remove_meas_and_barriers('./circuits/')

Let's first see how many single-qubit and two-qubit gates the circuit has, and how long would it take to execute on a trapped-ion hardware in which single-qubit gates take $110\mu s$ and two-qubit gates $672\mu s$.

In [3]:
t, sqgs, tqgs = calculate_circuit_runtime('./circuits/adder_n28_transpiled.qasm')
print('Number of SQGs:', sqgs)
print('Number of Entangling gates:', tqgs)
print('Execution time of quantum circuit:', str(t) + 'ms')

Number of SQGs: 1471
Number of Entangling gates: 195
Execution time of quantum circuit: 220.9ms


To compile the circuits using the linear programming + peephole method, we call the function `extract_peephole_ILP`. To compile using only the peephole rewrites we call `extract_peephole_only`. These functions return a PyZX `Circuit` class instance that we can later write again as a QASM file.

Note: there seems to be an issue with loading a local Gurobi installation through the Python-MIP package on jupyter notebooks, so in here we use a different solver (CBC) in the ILP part. If you have Gurobi installed in your machine, you can call these same functions within a python script instead to make use of Gurobi.

In [4]:
%%capture
# The extract functions receive a PyZX graph, so we first load and simplify using the usual PyZX full_reduce.
_, g = load_circuit('./circuits/no_measurements', 'adder_n28_transpiled.qasm')
# We solve with the ILP method
compiled_ilp, _, _ = extract_peephole_ILP(g.copy())
# Now we save the circuit using the gms as a QASM operation instead of RXX gates
save_recompiled('./circuits', 'adder_n28_compiled_ilp.qasm', compiled_ilp)

In [5]:
# We solve with the peephole only method 
_, g = load_circuit('./circuits/no_measurements', 'adder_n28_transpiled.qasm')
compiled_peephole, _, _ = extract_peephole_only(g.copy())
# Now we save the circuit using the gms as a QASM operation instead of RXX gates
save_recompiled('./circuits', 'adder_n28_compiled_peephole.qasm', compiled_peephole)

In [6]:
# We also compile using Qiskit
qc = qiskit.qasm2.load('./circuits/no_measurements/adder_n28_transpiled.qasm', custom_instructions=qiskit.qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
result_qc = transpile(qc, basis_gates=['rx','rz','rxx'], optimization_level=3)
with open('./circuits/adder_n28_compiled_qiskit.qasm','w') as saved_circuit:
    saved_circuit.write(qiskit.qasm2.dumps(result_qc))

Now let's see the metrics for each of the compilation methods.

In [7]:
t_ilp, sqgs_ilp, tqgs_ilp = calculate_circuit_runtime('./circuits/recompiled/adder_n28_compiled_ilp.qasm')
t_peephole, sqgs_peephole, tqgs_peephole = calculate_circuit_runtime('./circuits/recompiled/adder_n28_compiled_peephole.qasm')
t_qiskit, sqgs_qiskit, tqgs_qiskit = calculate_circuit_runtime('./circuits/adder_n28_compiled_qiskit.qasm')

print('Original -- SQGs:', sqgs, '- Entangling:', tqgs, '- Circuit runtime:', str(t) + 'ms')
print('ILP      -- SQGs: ', sqgs_ilp, '- Entangling: ', tqgs_ilp, '- Circuit runtime:', str(t_ilp) + 'ms')
print('Peephole -- SQGs: ', sqgs_peephole, '- Entangling: ', tqgs_peephole, '- Circuit runtime:', str(t_peephole) + 'ms')
print('Qiskit   -- SQGs: ', sqgs_qiskit, '- Entangling:', tqgs_qiskit, '- Circuit runtime:', str(t_qiskit) + 'ms')

Original -- SQGs: 1471 - Entangling: 195 - Circuit runtime: 220.9ms
ILP      -- SQGs:  916 - Entangling:  77 - Circuit runtime: 79.0ms
Peephole -- SQGs:  863 - Entangling:  91 - Circuit runtime: 90.9ms
Qiskit   -- SQGs:  710 - Entangling: 195 - Circuit runtime: 176.6ms


Interestingly, using Gurobi in the ILP case for this circuit returns a circuit with also 77 GMS gates, but a lower single-qubit gate count. To reproduce the results of the paper for the ILP case, using Gurobi is then required.

If one wants to get a QASM circuit back with layers of RXX gates instead of the GMS custom instruction, simply write the output of `compiled_ilp.to_qasm()` or `compiled_peephole.to_qasm()` into a file.

For any questions feel free to contact `a.d.villoria.gonzalez@liacs.leidenuniv.nl`